# Simulation of quantum systems

In this notebook, we introduce the concepts of quantum simulation and the basic usage of SimuQ. 

Given a Hamiltonian $H$, the time evolution of a quantum system is governed by the Schrödinger equation. $$i\frac{d}{dt}|\psi(t)\rangle = H|\psi(t)\rangle.$$

Simulation of quantum systems is to solve the Schrödinger equation and obtain the system's state at a given time.

Due to the exponentially-large Hilbert space, a classical computer can not effectively simulate a quantum system's time evolution in general. SimuQ provides a more general and flexible way to simulate the time evolution of a quantum system using quantum computers.



## Installation

You may install SimuQ directly via `pip`:

In [1]:
%pip install simuq

Note: you may need to restart the kernel to use updated packages.


We start with programming the Ising model on a 3 qubit chain.

# Prepare the python environment


In [1]:
from simuq.qsystem import QSystem
from simuq.environment import Qubit

Here `QSystem` is the class for quantum systems, and `qubit` is the class for qubit sites.

## Define the evolution

First we create a quantum system and a list of qubit sites.

In [2]:
n_qubits = 3
qs = QSystem()
q = [Qubit(qs) for i in range(n_qubits)]

Suppose our target is a short evolution governed by an constant Ising Hamiltonian $H=X_1X_2+X_2X_3+Z_1+Z_2+Z_3$. We can program the evolution as follows.

In [3]:
h = q[0].X * q[1].X + q[1].X * q[2].X + q[0].Z + q[1].Z + q[2].Z

We add a $T=1$ time evolution under $H$ to the quantum system.


In [4]:
T = 1
qs.add_evolution(h, T)

Then `qs` contains the evolution of $H$ for time $T$. To add next segment of evolution, we only need to call the method `add_evolution` again.

# Create a QuTiP provider

A provider is a user interface for convenient manipulations of functionalities of SimuQ. QuTiP is a python package for simulating the dynamics of open quantum systems. We use QuTiP provider as a basic example on how to use providers to deploy quantum simulation problems on devices and obtain results.

We can create a QuTiP provider via the following code


In [6]:
from simuq.qutip import QuTiPProvider
qpp = QuTiPProvider()

## Compilation in provider

To simulate a quantum system `qs` programmed in HML via SimuQ, we need three major steps of a provider: `compile`, `run`, `results`.

We call the `compile` function of the provider to process the system into a runnable executable. For QuTiP provider, we can execute

In [7]:
qpp.compile(qs)

Compiled.


QuTiP provider processes the quantum system `qs` and translate it into a Hamiltonian in QuTiP. 

For other providers, compile command may specify the backend device, AAIS, and compiler specifications.

When compilation succeeds, the job will be recorded in the provider.

## Run and obtain results from providers

Running a job will send the compilation results to backend devices to execute. For QuTiP provider, we execute


In [8]:
qpp.run()

Solved.


For other providers, you can choose whether to run on classical simulators, and how many shots to run on the real device or simulators. Note that these classical simulators are mostly from hardware providers and executed on the cloud.

To retrieve the results, we can execute

In [9]:
qpp.results()

{'000': 0.697205360082755,
 '001': 0.0,
 '010': 0.0,
 '011': 0.05973358916350665,
 '100': 0.0,
 '101': 0.18332746159023183,
 '110': 0.05973358916350665,
 '111': 0.0}

If the task is still in the queue on the device, an exception will be raised. When the retrieval succeeds, a dictionary is returned, which contains the frequencies of obtaining a measurement array (encoded as a 0/1 string). A bit in the string corresponds to a site of the quantum system. 

We can call the following code to show the order of the sites in the measurement output.

In [10]:
qpp.print_sites()

Order of sites: ['Qubit0', 'Qubit1', 'Qubit2']


# Time dependent simulation
SimuQ also support simulating time dependent Hamiltonians. We use quantum annealing as an example to show how to program a time dependent Hamiltonian.

In quantum annealing, the Hamiltonian is a linear interpolation between an initial Hamiltonian $H_0$ and a target Hamiltonian $H_1$. $$H(t)=(1-\frac{t}{T})H_0+\frac{t}{T}H_1.$$ So that the state will evolve from the ground state of $H_0$ to the ground state of $H_1$.

In the following example, $H_0$ is chosen to be single qubit $X$ operators on each site, and $H_1$ is chosen to be the $ZZ$ interaction between neighboring sites in a ring.

In [32]:
import numpy as np

from simuq.qsystem import QSystem


def anneal(h0, h1, T):
    def f(t):
        return (1 - t / T) * h0 + t / T * h1

    return f


n = 5  # num of qubits
m = 10  # discretization
T = 5  # evolution time

qs = QSystem()
q = [Qubit(qs)] * n
h0, h1 = 0, 0
for i in range(n):
    h0 += q[i].X
for i in range(n):
    h1 += q[i % n].Z * q[(i + 1) % n].Z

qs.add_td_evolution(anneal(h0, h1, T), np.linspace(0, T, m))


Next, we use qutip provider to run the simulation.

In [33]:
from simuq.qutip import QuTiPProvider
qpp = QuTiPProvider()

Before we run the simulation, we need to prepare the initial state to the ground state of $H_0$, which is $|-\rangle^{\otimes N}$

In [34]:
from qutip import basis,tensor
ket = (basis(2, 0) - basis(2, 1)).unit()
initial_state=tensor([ket] * n)

Next, we compile and run with the given initial state.

In [35]:
qpp.compile(qs, initial_state=initial_state)
qpp.run()
results = qpp.results()

Compiled.
Solved.


To understant the quality of the result, we calculated the average energy.

In [37]:
def calc_average_energy(results):
    energy_avg = 0
    for result in results:
        energy=0
        for i in range(n):
            if result[i] != result[(i + 1) % n]:
                energy -= 1
            else:
                energy += 1
        energy_avg+=energy * results[result]
    return energy_avg

calc_average_energy(results)

-2.9713061671311745

# Non qubit systems

Thanks to the abstraction of a `site`, simuq can handle non-qubit systems as well with a uniform interface.

In [40]:
from simuq.environment import Fermion
from simuq.qsystem import QSystem
from simuq.transformation import jw_transform

alpha = 3
J = 1
D = 2

qs = QSystem()
f = [Fermion(qs)] * D

gamma_x = [f[i].a + f[i].c for i in range(D)]
gamma_y = [-1j * (f[i].a - f[i].c) for i in range(D)]

def model(alpha, J01, J12, J20):
    J = [[0, J01, -J20], [-J01, 0, J12], [J20, -J12, 0]]
    h = 0
    for i in range(D):
        h += 1j * alpha * gamma_x[i] * gamma_y[i]
    for i in range(D):
        for j in range(D):
            h += 0.5j * J[i][j] * gamma_x[i] * gamma_x[j]
    return h


qs.add_evolution(model(alpha, J, J, J), 1)
new_qs, new_sites = jw_transform(qs)

In [41]:
new_qs.evos[0][0].ham

[(['Z', ''], (-3+0j)), (['', 'Z'], (-3+0j)), (['Y', 'X'], (1-0j))]